In [163]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [164]:
#directories
KAGGLE_DIR='../input/'

In [165]:
train=pd.read_csv("../input/pubg-finish-placement-prediction/train_V2.csv")

In [166]:
test=pd.read_csv("../input/pubg-finish-placement-prediction/test_V2.csv")

In [167]:
train.head()

In [168]:
train.info()

In [169]:
train.shape

ILLEGAL MATCHES

In [170]:
train[train["winPlacePerc"].isnull()]

In [171]:
#by using inplace you dont have to create another variable to make that change.
train.drop(2744604,inplace=True)


In [172]:
import seaborn as sns
import matplotlib.pyplot as plt

# FEATURE ENGINEERING

Players Joined

In [173]:
train["playersJoined"]=train.groupby('matchId')['matchId'].transform('count')
plt.figure(figsize=(15,10))
sns.countplot(train[train['playersJoined']>=75]['playersJoined'])
plt.title('playersJoined')

# Normalization

Now that we have a feature 'playersJoined' we can normalize other features based on the amount of players. Features that can be valuable to normalize are:
1. kills
2. damageDealt
3. maxPlace
4. matchDuration

In [174]:
train['killsNorm']=train['kills']*((100-train['playersJoined'])/100+1)
train['damageDealtNorm']=train['damageDealt']*((100-train['playersJoined'])/100+1)
train['maxPlaceNorm']=train['maxPlace']*((100-train['playersJoined'])/100+1)
train['matchDurationNorm']=train['matchDuration']*((100-train['playersJoined'])/100+1)


In [175]:
train["totalDistance"]=train['rideDistance']+train['walkDistance']+train['swimDistance']

In [176]:
#creating a feature total distance of the player
train['totalDistance']
# creating a feature of kills without moving
train['killsWithoutMoving']=((train['kills']>0)&(train["totalDistance"]==0))

In [177]:
#data wh#players who are using mods and apk to win the game
display(train[train['killsWithoutMoving']==True].shape)

In [178]:
#dropping the data 
train.drop(train[train['killsWithoutMoving']==True].index,inplace=True)

In [179]:
train.shape

# HACK:ROAD KILLS

In [180]:
 train[train['roadKills']>10]

In [181]:
#drop road kill cheaters
train.drop(train[train['roadKills']>10].index ,inplace=True)

In [182]:
plt.figure(figsize=(15,5))
sns.countplot(data=train,x=train['kills'])
plt.show()


In [183]:
train[train['kills']>30].shape

In [184]:
train[train['kills']>30].head(5)

In [185]:
train.drop(train[train['kills']>30].index,inplace=True)

In [186]:
train.shape


# HACK HEADSHOT

In [187]:
train['headshot_rate']=train['headshotKills']/train['kills']
train['headshot_rate']=train['headshot_rate'].fillna(0)

In [188]:
plt.figure(figsize=(12,4))
sns.distplot(train['headshot_rate'])
plt.show()

In [189]:
display(train[(train['headshot_rate']==1) & (train['kills']>9)].shape)

In [190]:
(train[(train['headshot_rate']==1) & (train['kills']>9)]).head(10)

In [191]:
train.drop(train[train['headshot_rate']==1 & (train['kills']>9)].index,inplace=True)

In [192]:
train.shape

# HACK:LONGEST DISTANCE

In [193]:
train[train['longestKill']>=1000].shape

In [194]:
train.drop(train[train['longestKill']>=1000].index,inplace=True)

# HACK: WITHING DISTANCE

In [195]:
#if the player is travelling more distance by foot which is absurd
train[train['walkDistance']>=10000].shape

In [196]:
train.drop(train[train['walkDistance']>=10000].index,inplace=True)

In [197]:
#If the player is travelling more distance by vechile
# Remove outliers
train.drop(train[train['rideDistance'] >= 20000].index, inplace=True)

In [198]:
#if the player is swimming for more distance and which is absurd
train.drop(train[train['swimDistance']>=2000].index,inplace=True)

# HACK:WEOPENS USED

In [199]:
train.drop(train[train['weaponsAcquired']>=30].index, inplace = True)

# HACK:HEALS USED

In [200]:
train.drop(train[train['heals']>30].index)

In [201]:
cols=['kills','damageDealt','maxPlace','matchDuration']
train.drop(cols , axis=1,inplace=True)

In [202]:
train.shape

# Categorical data

In [203]:
train['matchType'].unique()

In [204]:
train['matchType'].nunique()

In [205]:
train=pd.get_dummies(train,columns=['matchType'])

In [206]:
train.head()

In [207]:
#turning match ids and group ids to categorical data
train['groupId']=train['groupId'].astype('category')
train['matchId']=train['matchId'].astype('category')

In [208]:
# Get category coding for groupId and matchID
train['groupId_cat'] = train['groupId'].cat.codes
train['matchId_cat'] = train['matchId'].cat.codes

In [209]:
#get rid of columns
train.drop(columns=['groupId','matchId'],inplace=True)

In [210]:
train.shape

In [211]:
#Lets take a look at our newly created features
train[['groupId_cat', 'matchId_cat']].head()

In [212]:
train.head()

In [213]:
train.drop(columns=['Id'],inplace=True)

In [214]:
train.head()

In [215]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [216]:
train.head()

In [217]:
train['winPlacePerc']

In [222]:
output_col=train['winPlacePerc']

In [219]:
output_col.head()

In [229]:
train.drop(['winPlacePerc'],axis=1)

In [238]:
x_train,x_test,y_train,y_test=train_test_split(train,output_col,test_size=0.2)

In [239]:
x_test

In [240]:
y_test.shape

In [241]:
y_train.shape

In [242]:
lr= LinearRegression()

In [243]:
lr.fit(x_train,y_train)

In [244]:
lr.coef_

In [245]:
lr.intercept_

# PREDICTIONS

In [246]:
predicted_value=lr.predict(x_test)

In [249]:
predicted_value

In [250]:
from sklearn.metrics import mean_squared_error

mean_squared_error(predicted_value,y_test)